In [1]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector import connect,DictCursor
import pandas as pd
import requests
import json

In [2]:
# api_key = "6567132011mshed68849c12b0db0p10f15ejsnd58ad5ce4767"  # Aksh
# api_key = "4f16ab08e8msh27e320e3d194653p1e0308jsn539b154232c5" # Divy
api_key = "8620937591msh4e8402b095c1122p1aa86cjsncc3f82db766b" # Ikram

In [3]:
headers = {
            "x-rapidapi-key": f"{api_key}",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }

http_url = "https://cricbuzz-cricket.p.rapidapi.com"

In [4]:
conn = snowflake.connector.connect(
 user='project',
 password='Project@123456',
 account='fzkreem-ojb05768',
 warehouse = 'COMPUTE_WH',
 role = 'ACCOUNTADMIN',
)

In [5]:
cs = conn.cursor()
# cs.execute('CREATE or REPLACE DATABASE "cricbuzz"')
cs.execute('USE DATABASE "cricbuzz"')

In [6]:
# cs.execute('CREATE OR REPLACE TABLE "cricbuzz_table" ("id" number(10),"rank" number(10),"name" varchar(50),'
# '"country" varchar(20),"rating" number(5),"difference" number(10,2),"points" number(5),"lastUpdatedOn" timestamptz,"trend" varchar(5),'
# '"faceImageId" number(10),"countryId" number(10),"isWomen" varchar(10),"formatType" varchar(10),"category" varchar(20))')

# cs.execute('USE TABLE "cricbuzz_table"')
cs.execute('SHOW TABLES')
TABLES = cs.fetchall()
TABLES

[(datetime.datetime(2025, 3, 12, 5, 44, 37, 348000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PLAYER_BATTING_STATS',
  'cricbuzz',
  'PUBLIC',
  'TABLE',
  '',
  '',
  108,
  8704,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  None,
  'N',
  'N',
  'N',
  'N'),
 (datetime.datetime(2025, 3, 12, 6, 23, 23, 854000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PLAYER_BOWLING_STATS',
  'cricbuzz',
  'PUBLIC',
  'TABLE',
  '',
  '',
  108,
  9216,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  None,
  'N',
  'N',
  'N',
  'N'),
 (datetime.datetime(2025, 3, 11, 6, 41, 16, 976000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'cricbuzz_table',
  'cricbuzz',
  'PUBLIC',
  'TABLE',
  '',
  '',
  225,
  10752,
  'ACCOUNTADMIN',
  '1',
  'OFF',
  'OFF',
  'OFF',
  None,
  None,
  'N',
  'N',
  'ROLE',
  'N',
  None,

In [ ]:
def get_icc_ranking_player():
    # l = [[0, 'test'], [0, 'odi'], [0, 't20'],[1,'test'], [1, 'odi'], [1, 't20']]
    # l = [[1,'test'],[1,'odi']]
    l = [
        ['batsmen', 'odi', 0],
        ['batsmen', 'test', 0],
        ['batsmen', 't20', 0],
        ['bowlers', 'odi', 0],
        ['bowlers', 'test', 0],
        ['bowlers', 't20', 0],
        ['allrounders', 'odi', 0],
        ['allrounders', 'test', 0],
        ['allrounders', 't20', 0],
        # ['teams', 'odi', 0],
        # ['teams', 'test', 0],
        # ['teams', 't20', 0],
        ['batsmen', 'odi', 1],
        # ['batsmen', 'test', 1],
        ['batsmen', 't20', 1],
        ['bowlers', 'odi', 1],
        # ['bowlers', 'test', 1],
        ['bowlers', 't20', 1],
        ['allrounders', 'odi', 1],
        # ['allrounders', 'test', 1],
        ['allrounders', 't20', 1],
        # ['teams', 'odi', 1],
        # ['teams', 'test', 1],
        # ['teams', 't20', 1]
    ]

    for i in l:
        url = f"{http_url}/stats/v1/rankings/{i[0]}?formatType={i[1]}&isWomen={int(i[2])}"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Check for HTTP errors
            data = response.json()
            
            df = pd.DataFrame(data['rank'])
            df['category'] = i[0]
            df['formatType'] = i[1]
            df['isWomen'] = i[2]

            write_pandas(conn, df, table_name="cricbuzz_table")
            
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            continue
        except KeyError as e:
            print(f"Key error: {e}")
            continue
        except Exception as e:
            print(f"An error occurred: {e}")
            continue

In [ ]:
# get_icc_ranking_player()

Request failed: Expecting value: line 1 column 1 (char 0)
Request failed: Expecting value: line 1 column 1 (char 0)
Request failed: Expecting value: line 1 column 1 (char 0)


In [9]:
curr = conn.cursor(DictCursor)
curr.execute('SELECT * FROM "cricbuzz_table"')
data_table = curr.fetchall()
ranking_data = pd.DataFrame(data_table)

In [11]:
ranking_data

,id,rank,name,country,rating,difference,points,lastUpdatedOn,trend,faceImageId,countryId,isWomen,formatType,category
0,11808,1,Shubman Gill,India,791,1.00,791,2025-03-11 00:00:00-07:00,Up,616515,2,0,odi,batsmen
1,8359,2,Babar Azam,Pakistan,770,-1.00,770,2025-03-11 00:00:00-07:00,Down,616440,3,0,odi,batsmen
2,10209,3,Heinrich Klaasen,South Africa,760,2.00,760,2025-03-11 00:00:00-07:00,Up,619866,11,0,odi,batsmen
3,1413,4,Virat Kohli,India,747,2.00,747,2025-03-11 00:00:00-07:00,Up,616517,2,0,odi,batsmen
4,576,5,Rohit Sharma,India,745,-2.00,745,2025-03-11 00:00:00-07:00,Down,616514,2,0,odi,batsmen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,9089,9,Deandra Dottin,West Indies,243,None,243,2025-03-08 00:00:00-08:00,New,227427,10,1,t20,allrounders
221,12752,12,Nadine de Klerk,South Africa,237,None,237,2025-03-08 00:00:00-08:00,Flat,586253,11,1,t20,allrounders
222,11531,13,Sophie Ecclestone,England,232,None,232,2025-03-08 00:00:00-08:00,Flat,255054,9,1,t20,allrounders
223,14806,14,Fatima Sana,Pakistan,213,None,213,2025-03-08 00:00:00-08:00,Flat,255051,3,1,t20,allrounders


In [16]:
write_pandas(conn,ranking_data,table_name="cricbuzz_table")

(True,
 1,
 225,
 [('lvjwqhuugs/file0.txt', 'LOADED', 225, 225, 1, 0, None, None, None, None)])

In [16]:
cs.execute('CREATE OR REPLACE TABLE "team_list" ("teamId" number(10),"teamName" varchar(50),"teamSName" varchar(20),"countryName" varchar(20))')

In [17]:
def get_teams_data():   
     try:
          url = f"{http_url}/teams/v1/international"
          response = requests.get(url, headers=headers)
          response.raise_for_status() 
          data = response.json()
          
          for i in data['list']:
               if i['teamName'] == 'Associate Teams':
                    break 
               
               if len(i) < 2:
                    continue

               print(i)
               i.pop('imageId')
               df1 = pd.DataFrame([i])  # Corrected DataFrame creation
               write_pandas(conn, df1, table_name="team_list")

     except requests.exceptions.RequestException as e:
          print(f"Request failed: {e}")
     except Exception as e:
          print(f"An error occurred: {e}")

In [ ]:
get_teams_data()

In [ ]:
curr = conn.cursor(DictCursor)
curr.execute('SELECT * FROM "team_list"')
data_table = curr.fetchall()
pd.DataFrame(data_table)

In [40]:
cs.execute('CREATE OR REPLACE TABLE "team_players" ("id" number(10),"name" varchar(50),"battingStyle" varchar(50),"bowlingStyle" varchar(50),"role" varchar(20),"teamId" number(10))')

In [22]:
def get_temas_player():
    teamId = 4
    role = ''
    url = f"{http_url}/teams/v1/{teamId}/players"
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check for HTTP errors
        data = response.json()
        
        for i in data['player']:
            if i['name'] == 'BATSMEN':
                role = 'Batsman'
                continue
            elif i['name'] == 'ALL ROUNDER':
                role = 'Allrounder'
                continue
            elif i['name'] == 'WICKET KEEPER':
                role = 'WK-Batsman'
                continue
            elif i['name'] == 'BOWLER':
                role = 'Bowler'
                continue

            i['role'] = role
            i['teamId'] = teamId
            i.pop('imageId', None)
            print(i)
            
            df1 = pd.DataFrame([i])
            write_pandas(conn, df1, table_name="team_players")
            
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except KeyError as e:
        print(f"Key error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [23]:
get_temas_player()

{'id': '1643', 'name': 'Aaron Finch', 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Left-arm orthodox', 'role': 'Batsman', 'teamId': 4}
{'id': '2250', 'name': 'Steven Smith', 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak', 'role': 'Batsman', 'teamId': 4}
{'id': '1739', 'name': 'David Warner', 'battingStyle': 'Left-hand bat', 'role': 'Batsman', 'teamId': 4}
{'id': '10182', 'name': 'Marnus Labuschagne', 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak', 'role': 'Batsman', 'teamId': 4}
{'id': '8121', 'name': 'Joe Burns', 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium', 'role': 'Batsman', 'teamId': 4}
{'id': '12228', 'name': 'Will Pucovski', 'battingStyle': 'Right-hand bat', 'role': 'Batsman', 'teamId': 4}
{'id': '8497', 'name': 'Travis Head', 'battingStyle': 'Left-hand bat', 'bowlingStyle': 'Right-arm offbreak', 'role': 'Batsman', 'teamId': 4}
{'id': '7662', 'name': 'Glenn Maxwell', 'battingStyle': 'Right-hand bat', 'b

In [29]:
curr = conn.cursor(DictCursor)
curr.execute('SELECT * FROM team_players_main where teamId = 2')
data_table = curr.fetchall()
team_p = pd.DataFrame(data_table)

In [ ]:
# url = f"{http_url}/stats/v1/player/1413/batting"
# response = requests.get(url, headers=headers)
# response.raise_for_status()  # Check for HTTP errors
# data = response.json()


In [30]:
# data
team_p

,ID,NAME,BATTINGSTYLE,BOWLINGSTYLE,ROLE,TEAMID
0,1413,Virat Kohli,Right-hand bat,Right-arm medium,Batsman,2
1,576,Rohit Sharma,Right-hand bat,Right-arm offbreak,Batsman,2
2,9311,Jasprit Bumrah,Right-hand bat,Right-arm fast,Bowler,2
3,1448,Cheteshwar Pujara,Right-hand bat,Right-arm legbreak,Batsman,2
4,7910,Yuzvendra Chahal,Right-hand bat,Right-arm legbreak,Bowler,2
5,1836,Manish Pandey,Right-hand bat,Right-arm medium,Batsman,2
6,587,Ravindra Jadeja,Left-hand bat,Left-arm orthodox,Allrounder,2
7,8424,Hanuma Vihari,Right-hand bat,Right-arm offbreak,Allrounder,2
8,7909,Mohammed Shami,Right-hand bat,Right-arm fast,Bowler,2
9,10808,Mohammed Siraj,Right-hand bat,Right-arm fast,Bowler,2


In [129]:
# create_table_query = """
# CREATE or replace TABLE PLAYER_BATTING_STATS (
#     PlayerID NUMBER(10, 0),
#     PlayerName VARCHAR(20),
#     Format VARCHAR(20),
#     Matches NUMBER(10, 0),
#     Innings NUMBER(10, 0),
#     Runs NUMBER(10, 0),
#     Balls NUMBER(10, 0),
#     Highest NUMBER(10, 0),
#     Average NUMBER(10, 2),
#     SR NUMBER(10, 2),
#     NotOut NUMBER(10, 0),
#     Fours NUMBER(10, 0),
#     Sixes NUMBER(10, 0),
#     Ducks NUMBER(10, 0),
#     Fifties NUMBER(10, 0),
#     Hundreds NUMBER(10, 0),
#     DoubleHundreds NUMBER(10, 0),
#     TripleHundreds NUMBER(10, 0),
#     QuadrupleHundreds NUMBER(10, 0)
# )
# """
# cs.execute(create_table_query)

In [ ]:
# create_table_query = """
# CREATE or replace TABLE PLAYER_BOWLING_STATS (
#     PlayerID NUMBER(10, 0),
#     PlayerName VARCHAR(20),
#     Format VARCHAR(20),
#     Matches NUMBER(10, 0),
#     Innings NUMBER(10, 0),
#     Balls NUMBER(10, 0),
#     Runs NUMBER(10, 0),
#     Maidens NUMBER(10, 0),
#     Wickets NUMBER(10, 0),
#     Avg NUMBER(10,2),
#     Eco NUMBER(10,2),
#     SR NUMBER(10, 2),
#     BBI VARCHAR(20),
#     BBM VARCHAR(20),
#     FourW NUMBER(10,0),
#     FiveW NUMBER(10,0),
#     TenW NUMBER(10,0)
# )
# """
# cs.execute(create_table_query)

In [140]:
import time

def fetch_player_data(player_id):
    url = f"{http_url}/stats/v1/player/{player_id}/bowling"
    
    # response = requests.get(url, headers=headers)
    # response.raise_for_status()  
    # return response.json()
    
    retries = 5
    for i in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 503:
            wait_time = 2 ** i  # Exponential backoff
            print(f"Service unavailable. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            response.raise_for_status()  # Check for other HTTP errors
            return response.json()
    raise Exception("Failed to fetch data after multiple retries")

def process_data(data, player_id, player_name):
    headers = data['headers'][1:]  # Skip 'ROWHEADER'
    values = data['values']
    
    records = []
    for i, format in enumerate(headers):
        record = {}
        
        record['PlayerID'] = player_id
        record['Format'] = format
        record['PlayerName'] = player_name
        
        for _,val in enumerate(values):
            col_name = val['values'][0]
            # record[col_name] = float(val['values'][i+1]) if '.' in val['values'][i+1] else int(val['values'][i+1])
            if '.' in val['values'][i+1] :
                record[col_name] = float(val['values'][i+1])
            elif '/' in val['values'][i + 1]:
                record[col_name] = str(val['values'][i+1])
            else :
                record[col_name] = int(val['values'][i+1])
                
        records.append(record)
        
    return records

In [ ]:
# all_records = []

# for player_id in team_p['id']:
#     data = fetch_player_data(player_id)
#     player_name = team_p.loc[team_p['id'] == player_id,'name'].values[0]
#     records = process_data(data, player_id, player_name)
#     all_records.extend(records)
    
# player_stats_df = pd.DataFrame(all_records)
# print(player_stats_df.to_string())
    
# for index, row in player_stats_df.iterrows():
#     insert_query = f"""
#     INSERT INTO PLAYER_BATTING_STATS (
#         PlayerID, PlayerName, Format, Matches, Innings, Runs, Balls, Highest, Average, SR, NotOut, Fours, Sixes, Ducks, Fifties, Hundreds, DoubleHundreds, TripleHundreds, QuadrupleHundreds
#     ) VALUES (
#         {row['PlayerID']}, '{row['PlayerName']}', '{row['Format']}', {row['Matches']}, {row['Innings']}, {row['Runs']}, {row['Balls']}, {row['Highest']}, {row['Average']}, {row['SR']}, {row['Not Out']}, {row['Fours']}, {row['Sixes']}, {row['Ducks']}, {row['50s']}, {row['100s']}, {row['200s']}, {row['300s']}, {row['400s']}
#     )
#     """
# cs.execute(insert_query)

In [141]:
all_records = []

for player_id in team_p['id']:
    data = fetch_player_data(player_id)
    player_name = team_p.loc[team_p['id'] == player_id,'name'].values[0]
    records = process_data(data, player_id, player_name)
    all_records.extend(records)
    
player_stats_df = pd.DataFrame(all_records)
print(player_stats_df.to_string())
    
for index, row in player_stats_df.iterrows():
    insert_query = f"""
    INSERT INTO PLAYER_BOWLING_STATS (
        PlayerID, PlayerName, Format, Matches, Innings, Balls, Runs, Maidens, Wickets,Avg,Eco, SR, BBI, BBM , FourW , FiveW , TenW
    ) VALUES (
        {row['PlayerID']}, '{row['PlayerName']}', '{row['Format']}', {row['Matches']}, {row['Innings']}, {row['Balls']},{row['Runs']}, {row['Maidens']}, {row['Wickets']}, {row['Avg']}, {row['Eco']}, {row['SR']}, '{row['BBI']}', '{row['BBM']}', {row['4w']}, {row['5w']}, {row['10w']}
    )
    """
    cs.execute(insert_query)

     PlayerID Format           PlayerName  Matches  Innings  Balls   Runs  Maidens  Wickets     Avg    Eco      SR   BBI     BBM  4w  5w  10w
0        1413   Test          Virat Kohli      123       11    175     84        2        0    0.00   2.88    0.00   0/0     0/0   0   0    0
1        1413    ODI          Virat Kohli      302       50    662    680        1        5  136.00   6.16  132.40  1/13    1/13   0   0    0
2        1413    T20          Virat Kohli      125       13    152    204        0        4   51.00   8.05   38.00  1/13    1/13   0   0    0
3        1413    IPL          Virat Kohli      252       26    251    368        0        4   92.00   8.80   62.75  2/25    2/25   0   0    0
4         576   Test         Rohit Sharma       67       16    383    224        5        2  112.00   3.51  191.50  1/26    1/35   0   0    0
5         576    ODI         Rohit Sharma      273       40    610    533        2        9   59.22   5.24   67.78  2/27    2/27   0   0    0
6     

In [142]:
# curr = conn.cursor(DictCursor)
# curr.execute('SELECT * FROM PLAYER_BATTING_STATS')
# data_table = curr.fetchall()
# player_batting = pd.DataFrame(data_table)


curr = conn.cursor(DictCursor)
curr.execute('SELECT * FROM PLAYER_BOWLING_STATS')
data_table = curr.fetchall()
player_batting = pd.DataFrame(data_table)

In [143]:
print(player_batting.to_string())

     PLAYERID           PLAYERNAME FORMAT  MATCHES  INNINGS  BALLS   RUNS  MAIDENS  WICKETS     AVG    ECO      SR   BBI     BBM  FOURW  FIVEW  TENW
0        1413          Virat Kohli   Test      123       11    175     84        2        0    0.00   2.88    0.00   0/0     0/0      0      0     0
1        1413          Virat Kohli    ODI      302       50    662    680        1        5  136.00   6.16  132.40  1/13    1/13      0      0     0
2        1413          Virat Kohli    T20      125       13    152    204        0        4   51.00   8.05   38.00  1/13    1/13      0      0     0
3        1413          Virat Kohli    IPL      252       26    251    368        0        4   92.00   8.80   62.75  2/25    2/25      0      0     0
4         576         Rohit Sharma   Test       67       16    383    224        5        2  112.00   3.51  191.50  1/26    1/35      0      0     0
5         576         Rohit Sharma    ODI      273       40    610    533        2        9   59.22   5.24